### I. Change infor of json file after annotating by labelme. 
##### We can skip this part if we dont need and move to part II
When we have many folders that images inside the folders are annotated. Then, we want to move images of all folders to one folders. 
We must change images name and json name. 

In [ ]:
import json
import os
import glob
import pprint

In [ ]:
def updateJsonFile(jsfile,user):
    jsonFile = open(jsfile, "r") # Open the JSON file for reading
    data = json.load(jsonFile) # Read the JSON into the buffer
    jsonFile.close() # Close the JSON file
    
    ## Working with buffered content
    
    data["imagePath"] = user + "_" + data["imagePath"]

    ## Save our changes to JSON file
    jsonFile = open(jsfile, "w+")
    jsonFile.write(json.dumps(data))
    jsonFile.close()

In [ ]:
fileList = []
keywordList = []
userID = "24491"
userPart = "1"
user = userID + userPart
path = "./Rightside_window_user_id_"+userID+"_NoAudio_"+ userPart +"/"

for filename in glob.glob(os.path.join(path, '*.jpg')):
    directory = str(os.path.dirname(os.path.abspath(filename)))
    filename = os.path.basename(filename)
    print("directory",directory)
    print("location",filename)
    source = directory + "/" + filename
    filenamenew = user + "_" + filename
    dest = directory + "/" + filenamenew
    os.rename(source,dest)
    print(filename)
for jsfile in glob.glob(os.path.join(path, '*.json')):
    directory = str(os.path.dirname(os.path.abspath(jsfile)))
    jsfile = os.path.basename(jsfile)
    print("directory",directory)
    print("location",jsfile)
    
    source = directory + "/" + jsfile
    jsfilenew = user + "_" + jsfile
    print("source",source)
    
    dest = directory + "/" + jsfilenew
    print("dest",dest)
    os.rename(source,dest)  
    
    updateJsonFile(dest,user)
    print(jsfile)


### II. After json files and image files are moved to one folder. We divide to three folders. 
+ train
+ val
+ test

In [ ]:
import os
from random import choice
import shutil
import json

#arrays to store file names
imgs =[]
jsons =[]

#setup dir names
trainPath = '../data/train'
valPath = '../data/val'
testPath = '../data/test'
crsPath = '../data/dataset' #dir where images and annotations stored

if not os.path.exists(trainPath):
    os.makedirs(trainPath)
if not os.path.exists(valPath):
    os.makedirs(valPath)
if not os.path.exists(testPath):
    os.makedirs(testPath)    
        
#setup ratio (val ratio = rest of the files in origin dir after splitting into train and test)
train_ratio = 0.8
test_ratio = 0.1


#total count of imgs
totalImgCount = len(os.listdir(crsPath))/2

# #soring files to corresponding arrays
for (dirname, dirs, files) in os.walk(crsPath):
    for filename in files:
        if filename.endswith('.json'):
            jsons.append(filename)
        else:
            imgs.append(filename)

print ("jsons len",len(jsons))
print ("imgs len",len(imgs))

#counting range for cycles
countForTrain = int(len(imgs)*train_ratio)
countForTest = int(len(imgs)*test_ratio)

#cycle for train dir
for x in range(countForTrain):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileJson = fileJpg[:-4] +'.json' # get name of corresponding annotation file
    print ("fileJpg",fileJpg)
    print ("fileJson",fileJson)

    #move both files into train dir
    shutil.move(os.path.join(crsPath, fileJpg), os.path.join(trainPath, fileJpg))
    shutil.move(os.path.join(crsPath, fileJson), os.path.join(trainPath, fileJson))

    #remove files from arrays
    imgs.remove(fileJpg)
    jsons.remove(fileJson)

#cycle for test dir   
for x in range(countForTest):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileJson = fileJpg[:-4] +'.json' # get name of corresponding annotation file

    #move both files into train dir
    shutil.move(os.path.join(crsPath, fileJpg), os.path.join(testPath, fileJpg))
    shutil.move(os.path.join(crsPath, fileJson), os.path.join(testPath, fileJson))

    #remove files from arrays
    imgs.remove(fileJpg)
    jsons.remove(fileJson)

#rest of files will be validation files, so rename origin dir to val dir
os.rename(crsPath, valPath)

#summary information after splitting
print('Total images: ', totalImgCount)
print('Images in train dir:', len(os.listdir(trainPath))/2)
print('Images in test dir:', len(os.listdir(testPath))/2)
print('Images in validation dir:', len(os.listdir(valPath))/2)

#### Join json file of 3 folder before split json and image separately 

In [ ]:

# labelme2coco val
# labelme2coco train
# labelme2coco test

In [ ]:
import os
os.listdir()

In each folder (train, val,test) we divide to two small folder named annotation_data and image_data

In [ ]:
#setup dir names
trainPath = os.path.abspath('../data/train')
valPath = os.path.abspath('../data/val')
testPath = os.path.abspath('../data/test')
crsPath = os.path.abspath('../data/dataset') #dir where images and annotations stored


def split_image_and_annotation(crsPath):
    ann_file = crsPath + '/annotation_data'
    img_prefix =crsPath + '/image_data'
    if not os.path.exists(ann_file):
        os.makedirs(ann_file)
    if not os.path.exists(img_prefix):
        os.makedirs(img_prefix)
    for (dirname, dirs, files) in os.walk(crsPath):
        print("len",len(files))
        for i in range(len(files)):
            print("i",i)
            filename = files[i]
            print("filename",filename)
            if filename.endswith('.json'):
                shutil.move(os.path.join(crsPath, filename), os.path.join(ann_file, filename))
            if filename.endswith('.jpg'):
                shutil.move(os.path.join(crsPath, filename), os.path.join(img_prefix, filename))
        return

In [ ]:
split_image_and_annotation(testPath)

### III Training 
#### Modify the config
In the next step, we need to modify the config for the training. To accelerate the process, we finetune a detector using a pre-trained detector.

In [1]:
import os 
os.chdir("../../mmdetection")
os.listdir()

['data',
 '.circleci',
 '.git',
 'demo',
 'checkpoints',
 '.readthedocs.yml',
 'setup.cfg',
 'docs',
 'model-index.yml',
 '.github',
 'README.md',
 'requirements.txt',
 'resources',
 '.gitignore',
 'CITATION.cff',
 'new_data_custom_config.py',
 '.owners.yml',
 '.pre-commit-config.yaml',
 'MANIFEST.in',
 'work_dirs',
 'LICENSE',
 'tests',
 'tutorial_exps',
 'configs',
 'mmdet',
 '.dev_scripts',
 'pytest.ini',
 'requirements',
 'README_zh-CN.md',
 'docker',
 'tools',
 'setup.py']

In [2]:
from mmcv import Config
cfg = Config.fromfile('configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')

/home/pidl/.local/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


#### Create a custom dataset that name is incar.py into mmdet/datasets
```
import contextlib
import io
import itertools
import logging
import os.path as osp
import tempfile
import warnings
from collections import OrderedDict

import mmcv
import numpy as np
from mmcv.utils import print_log
from terminaltables import AsciiTable

from mmdet.core import eval_recalls
from .api_wrappers import COCO, COCOeval
from .builder import DATASETS
from .custom import CustomDataset


# https://mmdetection.readthedocs.io/en/latest/tutorials/customize_dataset.html

@DATASETS.register_module()
class IncarDataset(CustomDataset):

    CLASSES = ('steering wheel', 'cell phone', 'hat', 'bottle', 'control panel')

    def load_annotations(self, ann_file):
        """Load annotation from COCO style annotation file.

        Args:
            ann_file (str): Path of annotation file.

        Returns:
            list[dict]: Annotation info from COCO api.
        """

        self.coco = COCO(ann_file)
        # The order of returned `cat_ids` will not
        # change with the order of the CLASSES
        self.cat_ids = self.coco.get_cat_ids(cat_names=self.CLASSES)

        self.cat2label = {cat_id: i for i, cat_id in enumerate(self.cat_ids)}
        self.img_ids = self.coco.get_img_ids()
        data_infos = []
        total_ann_ids = []
        for i in self.img_ids:
            info = self.coco.load_imgs([i])[0]
            info['filename'] = info['file_name']
            data_infos.append(info)
            ann_ids = self.coco.get_ann_ids(img_ids=[i])
            total_ann_ids.extend(ann_ids)
        assert len(set(total_ann_ids)) == len(
            total_ann_ids), f"Annotation ids in '{ann_file}' are not unique!"
        return data_infos

    def get_ann_info(self, idx):
        """Get COCO annotation by index.

        Args:
            idx (int): Index of data.

        Returns:
            dict: Annotation info of specified index.
        """

        img_id = self.data_infos[idx]['id']
        ann_ids = self.coco.get_ann_ids(img_ids=[img_id])
        ann_info = self.coco.load_anns(ann_ids)
        return self._parse_ann_info(self.data_infos[idx], ann_info)

    def _parse_ann_info(self, img_info, ann_info):
        """Parse bbox and mask annotation.

        Args:
            ann_info (list[dict]): Annotation info of an image.
            with_mask (bool): Whether to parse mask annotations.

        Returns:
            dict: A dict containing the following keys: bboxes, bboxes_ignore,\
                labels, masks, seg_map. "masks" are raw annotations and not \
                decoded into binary masks.
        """
        gt_bboxes = []
        gt_labels = []
        gt_bboxes_ignore = []
        gt_masks_ann = []
        for i, ann in enumerate(ann_info):
            if ann.get('ignore', False):
                continue
            x1, y1, w, h = ann['bbox']
            inter_w = max(0, min(x1 + w, img_info['width']) - max(x1, 0))
            inter_h = max(0, min(y1 + h, img_info['height']) - max(y1, 0))
            if inter_w * inter_h == 0:
                continue
            if ann['area'] <= 0 or w < 1 or h < 1:
                continue
            if ann['category_id'] not in self.cat_ids:
                continue
            bbox = [x1, y1, x1 + w, y1 + h]
            if ann.get('iscrowd', False):
                gt_bboxes_ignore.append(bbox)
            else:
                gt_bboxes.append(bbox)
                gt_labels.append(self.cat2label[ann['category_id']])
                gt_masks_ann.append(ann.get('segmentation', None))

        if gt_bboxes:
            gt_bboxes = np.array(gt_bboxes, dtype=np.float32)
            gt_labels = np.array(gt_labels, dtype=np.int64)
        else:
            gt_bboxes = np.zeros((0, 4), dtype=np.float32)
            gt_labels = np.array([], dtype=np.int64)

        if gt_bboxes_ignore:
            gt_bboxes_ignore = np.array(gt_bboxes_ignore, dtype=np.float32)
        else:
            gt_bboxes_ignore = np.zeros((0, 4), dtype=np.float32)

        seg_map = img_info['filename'].rsplit('.', 1)[0] + self.seg_suffix

        ann = dict(
            bboxes=gt_bboxes,
            labels=gt_labels,
            bboxes_ignore=gt_bboxes_ignore,
            masks=gt_masks_ann,
            seg_map=seg_map)

        return ann
```

#### Modify file __init__.py inside mmdet/datasets
```
# Copyright (c) OpenMMLab. All rights reserved.
from .builder import DATASETS, PIPELINES, build_dataloader, build_dataset
from .cityscapes import CityscapesDataset
from .coco import CocoDataset
from .coco_panoptic import CocoPanopticDataset
from .custom import CustomDataset
#*****Tommy*****#
from .incar import IncarDataset
#**************# Then insert 'IncarDataset' to __all__ 
from .dataset_wrappers import (ClassBalancedDataset, ConcatDataset,
                               MultiImageMixDataset, RepeatDataset)
from .deepfashion import DeepFashionDataset
from .lvis import LVISDataset, LVISV1Dataset, LVISV05Dataset
from .openimages import OpenImagesChallengeDataset, OpenImagesDataset
from .samplers import DistributedGroupSampler, DistributedSampler, GroupSampler
from .utils import (NumClassCheckHook, get_loading_pipeline,
                    replace_ImageToTensor)
from .voc import VOCDataset
from .wider_face import WIDERFaceDataset
from .xml_style import XMLDataset

__all__ = [
    'IncarDataset','CustomDataset', 'XMLDataset', 'CocoDataset', 'DeepFashionDataset',
    'VOCDataset', 'CityscapesDataset', 'LVISDataset', 'LVISV05Dataset',
    'LVISV1Dataset', 'GroupSampler', 'DistributedGroupSampler',
    'DistributedSampler', 'build_dataloader', 'ConcatDataset', 'RepeatDataset',
    'ClassBalancedDataset', 'WIDERFaceDataset', 'DATASETS', 'PIPELINES',
    'build_dataset', 'replace_ImageToTensor', 'get_loading_pipeline',
    'NumClassCheckHook', 'CocoPanopticDataset', 'MultiImageMixDataset',
    'OpenImagesDataset', 'OpenImagesChallengeDataset'
]
```

In [4]:
from mmdet.apis import set_random_seed
# CocoDataset
# IncarDataset
# Modify dataset type and path
cfg.dataset_type = 'IncarDataset'
cfg.data_root = 'data/incar/'

cfg.data.test.type = 'IncarDataset'
cfg.data.test.data_root = 'data/incar/'
cfg.data.test.ann_file = 'test/annotation_coco.json'
cfg.data.test.img_prefix = 'test/'

cfg.data.train.type = 'IncarDataset'
cfg.data.train.data_root = 'data/incar/'
cfg.data.train.ann_file = 'train/annotation_coco.json'
cfg.data.train.img_prefix = 'train/'

cfg.data.val.type = 'IncarDataset'
cfg.data.val.data_root = 'data/incar/'
cfg.data.val.ann_file = 'val/annotation_coco.json'
cfg.data.val.img_prefix = 'val'

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 5
# If we need to finetune a model based on a pre-trained detector, we need to
# use load_from to set the path of checkpoints.
cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = 'cuda'
cfg.gpu_ids = range(1)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='open-mmlab://detectron2/resnet50_caffe')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='Cro

#### Train a new detector

Finally, lets initialize the dataset and detector, then train a new detector! We use the high-level API train_detector implemented by MMDetection. This is also used in our training scripts. For details of the implementation, please see [here](https://github.com/open-mmlab/mmdetection/blob/master/mmdet/apis/train.py)



In [5]:
import os.path as osp

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]
datasets

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


/media/pidl/DataStore/Draft/SteeringWheelDataset/Working/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


[
 IncarDataset Train dataset with number of images 1752, and instance counts: 
 +--------------------+-------+----------------+-------+----------+-------+------------+-------+-------------------+-------+
 | category           | count | category       | count | category | count | category   | count | category          | count |
 +--------------------+-------+----------------+-------+----------+-------+------------+-------+-------------------+-------+
 | 0 [steering wheel] | 1752  | 1 [cell phone] | 1701  | 2 [hat]  | 426   | 3 [bottle] | 402   | 4 [control panel] | 31    |
 +--------------------+-------+----------------+-------+----------+-------+------------+-------+-------------------+-------+]

In [6]:
# Build the detector
model = build_detector(cfg.model)

# Build dataset

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir

if not os.path.exists(osp.abspath(cfg.work_dir)):
    os.makedirs(osp.abspath(cfg.work_dir))

train_detector(model, datasets, cfg, distributed=False, validate=True)


2022-12-01 19:19:54,011 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-01 19:19:54,020 - mmdet - INFO - load checkpoint from local path: checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth
2022-12-01 19:19:54,149 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([6, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([6]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([20, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2022-12-01 19:20:03,186 - mmdet - INFO - Epoch [1][10/876]	lr: 2.500e-03, eta: 2:30:45, time: 0.861, data_time: 0.236, memory: 3496, loss_rpn_cls: 0.0376, loss_rpn_bbox: 0.0161, loss_cls: 0.5985, acc: 83.9355, loss_bbox: 0.2810, loss: 0.9332
2022-12-01 19:20:08,599 - mmdet - INFO - Epoch [1][20/876]	lr: 2.500e-03, eta: 2:02:45, time: 0.543, data_time: 0.042, memory: 3496, loss_rpn_cls: 0.0297, loss_rpn_bbox: 0.0113, loss_cls: 0.2317, acc: 91.3965, loss_bbox: 0.3013, loss: 0.5740
2022-12-01 19:20:14,213 - mmdet - INFO - Epoch [1][30/876]	lr: 2.500e-03, eta: 1:54:24, time: 0.561, data_time: 0.041, memory: 3496, loss_rpn_cls: 0.0085, loss_rpn_bbox: 0.0120, loss_cls: 0.2136, acc: 94.1406, loss_bbox: 0.3031, loss: 0.5372
2022-12-01 19:20:19,909 - mmdet - INFO - Epoch [1][40/876]	lr: 2.500e-03, eta: 1:50:33, time: 0.569, data_time: 0.042, memory: 3496, loss_rpn_cls: 0.0030, loss_rpn_bbox: 0.0122, loss_cls: 0.1100, acc: 96.9824, loss_bbox: 0.2365, loss: 0.3618
2022-12-01 19:20:25,549 - mmdet 

2022-12-01 19:23:29,765 - mmdet - INFO - Epoch [1][350/876]	lr: 2.500e-03, eta: 1:44:07, time: 0.637, data_time: 0.046, memory: 3496, loss_rpn_cls: 0.0008, loss_rpn_bbox: 0.0050, loss_cls: 0.0412, acc: 98.4863, loss_bbox: 0.0920, loss: 0.1390
2022-12-01 19:23:36,182 - mmdet - INFO - Epoch [1][360/876]	lr: 2.500e-03, eta: 1:44:08, time: 0.638, data_time: 0.042, memory: 3496, loss_rpn_cls: 0.0006, loss_rpn_bbox: 0.0050, loss_cls: 0.0443, acc: 98.3789, loss_bbox: 0.1064, loss: 0.1563
2022-12-01 19:23:42,547 - mmdet - INFO - Epoch [1][370/876]	lr: 2.500e-03, eta: 1:44:07, time: 0.637, data_time: 0.047, memory: 3496, loss_rpn_cls: 0.0013, loss_rpn_bbox: 0.0067, loss_cls: 0.0459, acc: 98.2715, loss_bbox: 0.1051, loss: 0.1589
2022-12-01 19:23:48,766 - mmdet - INFO - Epoch [1][380/876]	lr: 2.500e-03, eta: 1:44:03, time: 0.622, data_time: 0.046, memory: 3496, loss_rpn_cls: 0.0004, loss_rpn_bbox: 0.0051, loss_cls: 0.0388, acc: 98.6230, loss_bbox: 0.0926, loss: 0.1369
2022-12-01 19:23:55,287 - mm

2022-12-01 19:27:07,086 - mmdet - INFO - Epoch [1][690/876]	lr: 2.500e-03, eta: 1:42:36, time: 0.635, data_time: 0.046, memory: 3496, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0046, loss_cls: 0.0271, acc: 99.0527, loss_bbox: 0.0753, loss: 0.1072
2022-12-01 19:27:13,345 - mmdet - INFO - Epoch [1][700/876]	lr: 2.500e-03, eta: 1:42:30, time: 0.625, data_time: 0.040, memory: 3496, loss_rpn_cls: 0.0005, loss_rpn_bbox: 0.0040, loss_cls: 0.0291, acc: 98.8770, loss_bbox: 0.0690, loss: 0.1026
2022-12-01 19:27:19,818 - mmdet - INFO - Epoch [1][710/876]	lr: 2.500e-03, eta: 1:42:26, time: 0.643, data_time: 0.040, memory: 3496, loss_rpn_cls: 0.0016, loss_rpn_bbox: 0.0044, loss_cls: 0.0380, acc: 98.6914, loss_bbox: 0.0852, loss: 0.1291
2022-12-01 19:27:26,012 - mmdet - INFO - Epoch [1][720/876]	lr: 2.500e-03, eta: 1:42:18, time: 0.619, data_time: 0.045, memory: 3496, loss_rpn_cls: 0.0003, loss_rpn_bbox: 0.0031, loss_cls: 0.0207, acc: 99.2480, loss_bbox: 0.0629, loss: 0.0871
2022-12-01 19:27:32,414 - mm

...

2022-12-01 21:02:11,534 - mmdet - INFO - Epoch [11][830/876]	lr: 2.500e-04, eta: 0:09:46, time: 0.656, data_time: 0.046, memory: 3496, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0015, loss_cls: 0.0148, acc: 99.4727, loss_bbox: 0.0273, loss: 0.0436
2022-12-01 21:02:17,917 - mmdet - INFO - Epoch [11][840/876]	lr: 2.500e-04, eta: 0:09:39, time: 0.645, data_time: 0.046, memory: 3496, loss_rpn_cls: 0.0000, loss_rpn_bbox: 0.0014, loss_cls: 0.0103, acc: 99.6289, loss_bbox: 0.0231, loss: 0.0349
2022-12-01 21:02:24,408 - mmdet - INFO - Epoch [11][850/876]	lr: 2.500e-04, eta: 0:09:33, time: 0.644, data_time: 0.039, memory: 3496, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0018, loss_cls: 0.0130, acc: 99.5215, loss_bbox: 0.0284, loss: 0.0432
2022-12-01 21:02:30,823 - mmdet - INFO - Epoch [11][860/876]	lr: 2.500e-04, eta: 0:09:27, time: 0.646, data_time: 0.044, memory: 3496, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0021, loss_cls: 0.0174, acc: 99.3848, loss_bbox: 0.0330, loss: 0.0525
2022-12-01 21:02:37,201 

2022-12-01 21:05:53,679 - mmdet - INFO - Epoch [12][300/876]	lr: 2.500e-05, eta: 0:06:06, time: 0.628, data_time: 0.042, memory: 3496, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0015, loss_cls: 0.0125, acc: 99.5020, loss_bbox: 0.0282, loss: 0.0422
2022-12-01 21:05:59,995 - mmdet - INFO - Epoch [12][310/876]	lr: 2.500e-05, eta: 0:05:59, time: 0.632, data_time: 0.047, memory: 3496, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0019, loss_cls: 0.0171, acc: 99.3652, loss_bbox: 0.0356, loss: 0.0547
2022-12-01 21:06:06,524 - mmdet - INFO - Epoch [12][320/876]	lr: 2.500e-05, eta: 0:05:53, time: 0.652, data_time: 0.047, memory: 3496, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0013, loss_cls: 0.0127, acc: 99.5605, loss_bbox: 0.0253, loss: 0.0394
2022-12-01 21:06:13,004 - mmdet - INFO - Epoch [12][330/876]	lr: 2.500e-05, eta: 0:05:47, time: 0.648, data_time: 0.048, memory: 3496, loss_rpn_cls: 0.0000, loss_rpn_bbox: 0.0016, loss_cls: 0.0111, acc: 99.5801, loss_bbox: 0.0231, loss: 0.0358
2022-12-01 21:06:19,374 

2022-12-01 21:09:31,490 - mmdet - INFO - Epoch [12][640/876]	lr: 2.500e-05, eta: 0:02:30, time: 0.631, data_time: 0.037, memory: 3496, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0014, loss_cls: 0.0099, acc: 99.5801, loss_bbox: 0.0231, loss: 0.0344
2022-12-01 21:09:37,792 - mmdet - INFO - Epoch [12][650/876]	lr: 2.500e-05, eta: 0:02:23, time: 0.634, data_time: 0.045, memory: 3496, loss_rpn_cls: 0.0000, loss_rpn_bbox: 0.0012, loss_cls: 0.0098, acc: 99.6289, loss_bbox: 0.0201, loss: 0.0311
2022-12-01 21:09:44,047 - mmdet - INFO - Epoch [12][660/876]	lr: 2.500e-05, eta: 0:02:17, time: 0.630, data_time: 0.041, memory: 3496, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0014, loss_cls: 0.0098, acc: 99.6191, loss_bbox: 0.0203, loss: 0.0315
2022-12-01 21:09:50,338 - mmdet - INFO - Epoch [12][670/876]	lr: 2.500e-05, eta: 0:02:10, time: 0.621, data_time: 0.037, memory: 3496, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0014, loss_cls: 0.0103, acc: 99.6582, loss_bbox: 0.0209, loss: 0.0326
2022-12-01 21:09:56,791 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 219/219, 6.6 task/s, elapsed: 33s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-01 21:12:36,215 - mmdet - INFO - 
+----------------+-----+------+--------+-------+
| class          | gts | dets | recall | ap    |
+----------------+-----+------+--------+-------+
| steering wheel | 219 | 219  | 1.000  | 1.000 |
| cell phone     | 215 | 217  | 1.000  | 1.000 |
| hat            | 58  | 56   | 0.000  | 0.000 |
| bottle         | 55  | 64   | 0.000  | 0.000 |
| control panel  | 2   | 3    | 0.500  | 0.500 |
+----------------+-----+------+--------+-------+
| mAP            |     |      |        | 0.500 |
+----------------+-----+------+--------+-------+
2022-12-01 21:12:36,289 - mmdet - INFO - Epoch(val) [12][219]	AP50: 0.5000, mAP: 0.5000


In [ ]:
pip install mmcv